In [38]:
from kafka import KafkaConsumer
from time import sleep 
from json import dumps, loads
import json
from s3fs import S3FileSystem
from datetime import datetime
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

In [39]:
# Declare Kafka Consumer
consumer = KafkaConsumer(
    'demo_test',
    bootstrap_servers=["ec2-44-206-255-156.compute-1.amazonaws.com:9092"],
    value_deserializer = lambda x: loads(x.decode('utf-8')))

In [42]:
# Create S3 Object 
s3 = S3FileSystem() 

# Intialize a list to hold the data 
tmp_data = [] 

for count, message in enumerate(consumer): 
    # Append the data to the list 
    tmp_data.append(message.value)
    # If we collect 10 messages save it to a parquet file
    if count != 0 and count % 10 == 0: 
        # Create a pandas DataFrame from the accumulated data
        df = pd.DataFrame(tmp_data)

        # Convert the DataFrame to a PyArrow table
        table = pa.Table.from_pandas(df)
        
        # Record current time to use in file name 
        current_timestamp = datetime.now().strftime("%Y%m%d_%H%M%S%Z")
        with s3.open(f"s3://kafka-stock-data-trial/stock_market_data_\
        {current_timestamp}.parquet", "wb") as file:
            print(table)
            pq.write_table(table, file)    
            #Renew the list 
            tmp_data = []

# If there are remaining rows that haven't been written to a file
if tmp_data:
    df = pd.DataFrame(tmp_data)
    table = pa.Table.from_pandas(df)
    current_timestamp = datetime.now().strftime("%Y%m%d_%H%M%S%Z")
    file_name = f"stock_market_data_{current_timestamp}.parquet"
    with s3.open(f"s3://kafka-stock-data-trial/{file_name}", "wb") as file:
        pq.write_table(table, file)

pyarrow.Table
Index: string
Date: string
Open: double
High: double
Low: double
Close: double
Adj Close: double
Volume: double
CloseUSD: double
----
Index: [["SSMI","N225","GSPTSE","GSPTSE","GDAXI",...,"NYA","HSI","N100","TWII","000001.SS"]]
Date: [["2004-12-02","1968-06-25","2015-12-31","1981-05-19","2012-05-02",...,"2020-11-27","1989-08-03","2001-03-26","2015-08-06","2011-08-17"]]
Open: [[5526,1501.709961,13077.59961,2332.300049,6861.299805,...,14191.58008,2634.600098,847.950012,8564.549805,2603.960938]]
High: [[5571.5,1501.709961,13098.59961,2332.300049,6875.870117,...,14229.12012,2634.600098,868.919983,8586.240234,2617.165039]]
Low: [[5506.5,1501.709961,12988,2315.100098,6664.569824,...,14165.70996,2634.600098,846.950012,8404.669922,2596.039063]]
Close: [[5569.299805,1501.709961,13010,2319.100098,6710.77002,...,14198.5,2634.600098,868.780029,8449.55957,2601.260986]]
Adj Close: [[5569.299805,1501.709961,13010,2313.210938,6710.77002,...,14198.5,2634.600098,868.780029,8449.527344,2601.

KeyboardInterrupt: 